IPOPT Tutorial Example

Victor M. Zavala 2016 

University of Wisconsin-Madison  

In [64]:
using JuMP 
using Ipopt 

In [65]:
# Model 
m = Model(solver=IpoptSolver(tol = 1e-8, max_iter = 100,linear_solver ="mumps",mu_strategy="monotone"))

NS=3
S=collect(1:NS)
NT=100
Time=collect(1:NT)
Timem=collect(1:NT-1)

Tf=10.0
dt=Tf/NT

Cai=zeros(NS)
pr=zeros(NS)

Cai[1]=15
Cai[2]=10
Cai[3]=20
pr[1]=0.7
pr[2]=0.2
pr[3]=0.1

Ti=20
Tci=15
rol= 800.8
cel=0.968
ror=1041.1
cer=1.291
U=6129
Ea=6867.36
R=8.314
Beta=5.967
H=69710.5
V=38.3
A=24;

@variable(m,  0.5 <= q[Time] <= 15, start= 2.83)    
@variable(m, 20 <= T[S,Time] <= 200, start = 73.3)
@variable(m, 10 <= Tc[S,Time] <= 69, start =63.31)
@variable(m, 0.9 <= Ca[S,Time] <= 16, start = 1.866)
@variable(m, 5 <= Cb[S,Time] <= 30, start = 13.1339)
@variable(m, 20 <= F[S,Time] <= 600, start = 450)
@variable(m, 0.7 <= x[S,Time] <= 0.9, start = 0.87) 
@variable(m, k[S,Time], start = 0.54) 
@variable(m, cost[S]) ;

In [66]:
@NLconstraint(m, balca[s in S, t in Timem], 
    (q[t]*(Cai[s] - Ca[s,t]) - V*k[s,t]*Ca[s,t])*dt == V*(Ca[s,t+1]-Ca[s,t])) 

@NLconstraint(m, balcb[s in S, t in Timem], 
    (V*k[s,t]*Ca[s,t] - q[t]*Cb[s,t])*dt == V*(Cb[s,t+1]-Cb[s,t])) 

@NLconstraint(m, balcT[s in S, t in Timem], 
    q[t]*rol*cel*(Ti - T[s,t]) + V*k[s,t]*Ca[s,t]*H - (U*A)*(T[s,t]- Tc[s,t]) 
    == V*cel*rol*(T[s,t+1]-T[s,t])/dt) 

@NLconstraint(m, balcTc[s in S, t in Timem], 
    F[s,t]*ror*cer*(Tci - Tc[s,t]) + U*A*(T[s,t] - Tc[s,t]) 
    == (V*0.1)*ror*cer*(Tc[s,t+1]-Tc[s,t])/dt) 

@NLconstraint(m, arrhenius[s in S, t in Time], 
    k[s,t] - Beta*exp(-Ea/(R*(T[s,t]+273.15))) == 0) 

@NLconstraint(m, conv[s in S, t in Time], 
    Ca[s,t] - Cai[s]*(1-x[s,t]) == 0) 

@constraint(m, approach[s in S, t in Time], T[s,t] - Tc[s,t] >= 5) 
@constraint(m, restime1[t in Time], V <= 25*q[t]) 
@constraint(m, restime2[t in Time], 15*q[t] <= V) 

@constraint(m, initCa[s in S], Ca[s,1] == 1.86)
@constraint(m, initCb[s in S], Cb[s,1] == 13.3)
@constraint(m, initT[s in S],   T[s,1] == 73.3)
@constraint(m, initTc[s in S], Tc[s,1] == 63.31)

@NLconstraint(m, costeq[s in S], cost[s]
    ==sum(100*q[t]*Cb[s,t] - 80*q[t]*Cai[s] for t in Time)) 

@NLobjective(m, Max, sum(pr[s]*cost[s] for s in S))

In [67]:
# solve model and get solution
solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     7458
Number of nonzeros in inequality constraint Jacobian.:      800
Number of nonzeros in Lagrangian Hessian.............:     7140

Total number of variables............................:     2203
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     1900
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1803
Total number of inequality constraints...............:      500
        inequality constraints with only lower bounds:      300
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      200

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

In [68]:
print(dt)

0.1

In [69]:
println("Ca = ", getvalue(Ca), "\n T = ", getvalue(T))
println("\n q = ", getvalue(q), "\n F = ", getvalue(F))
println("\n cost = ", getvalue(cost), "\n Tc= ", getvalue(Tc))

Ca = Ca: 2 dimensions:
[1,:]
  [1,  1] = 1.8607459734254101
  [1,  2] = 1.8460303066482022
  [1,  3] = 1.8294903207931055
  [1,  4] = 1.8201787695304323
  [1,  5] = 1.8172404741711112
  [1,  6] = 1.8186059784203241
  [1,  7] = 1.8227932098316315
  [1,  8] = 1.8287198034642071
  [1,  9] = 1.8356255792257494
  [1, 10] = 1.8429855789796712
  [1, 11] = 1.8504107235218448
  [1, 12] = 1.8576957831213925
  [1, 13] = 1.8644115668686119
  [1, 14] = 1.8709998122332978
  [1, 15] = 1.8771395638974298
  [1, 16] = 1.882802583260849
  [1, 17] = 1.887986825040905
  [1, 18] = 1.892697074642195
  [1, 19] = 1.8969313570984891
  [1, 20] = 1.9007945128483357
  [1, 21] = 1.9042868109867945
  [1, 22] = 1.9074523216028385
  [1, 23] = 1.9103355052029256
  [1, 24] = 1.912955599652581
  [1, 25] = 1.9153473704714186
  [1, 26] = 1.9175394578352298
  [1, 27] = 1.9195573529964267
  [1, 28] = 1.9214240837089374
  [1, 29] = 1.923145780658481
  [1, 30] = 1.9247632810081052
  [1, 31] = 1.926273750162384
  [1, 32] = 1.92